In [ ]:
# VALÉRIO DE ARRUDA, M. (marcel.a@ufabc.edu.br)

#======================================================
#===========                    =======================
#=========== MAXIMUM LIKELIHOOD =======================
#===========                    =======================
#======================================================

# Directory indication
#import os
#os.getcwd()
#os.chdir("/Users/your_username")

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import datetime
from datetime import timedelta, timezone, datetime
from scipy import stats
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import pylab as py
import matplotlib.patches as mpatches

# Data reading (.csv)
name = '/content/S_124.csv' # OpenSesame/JATOS Table (.csv) path
df = pd.read_csv(name, header = 0)
df
# Document/participant ID
name_file = (name.split(".")[0])
name_file = name_file.replace("/content/", "")
name_file

# Data pre-processing
post_practice = df.response_resp_log.size - 70
error_practice = post_practice - 20
perc_answer = pd.crosstab(df.response_resp_log[post_practice:], df.tempo_log, margins=True, normalize = 'columns')
time_stm = np.array([500.0, 630.0, 793.5, 1000.0, 1260.0, 1587.0, 2000])
perc_long = perc_answer[0:1].values
perc_long = np.delete(perc_long,[7])

# SIGMOID CURVE ADJUSTMENT WITH MAXIMUM LIKELIHOOD =======================================================
def sigmoid(params):
    k = params[0]    # parameter a from the previous code
    x0 = params[1]   
    sd = params[2]   # parameter b from the previous code

    yPred = 1 / (1+ np.exp(-k*(time_stm-x0)))
            
    # Calculate negative log likelihood
    LL = -np.sum(stats.norm.logpdf(perc_long, loc=yPred, scale=sd))

    return(LL)

# Definition of initial parameters
# initParams = [k, inflection point, sd]
# I'm still trying to figure out what k and sd are.

# sd according to the reference code is the Standard Deviation.
# but here in practice it seems to be related to the slope of the fitted curve.

# here are the guesses for the initial parameters.
# the second parameter is a guess for the inflection point (I have used geometric or arithmetic mean)
initParams = [0.0, 1000, 0.005]

# minimization of the distance between the initial parameters and the parameters of the sigmoid obtained
results = minimize(sigmoid, initParams, method='Nelder-Mead')
print("Parametros Iniciais [k, ponto de inflexão, sd]:\n",results.x)
estParms = results.x

rng = np.arange(time_stm[0],time_stm[-1],0.1)

yOut = yPred = 1 / (1+ np.exp(-estParms[0]*(rng-estParms[1])))


# Bisection Point===========================================

pnt_bisec = round(estParms[1],4)

# Differential Threshold====================================

    #0,25
T1_ = estParms[1] - (np.log(3)/estParms[0])

    #0,75
T2_ = estParms[1] + (np.log(3)/estParms[0])

lim_dif_ = round(((T2_-T1_)/2),4)

# Weber's Ratio==============================================

WR = round(((T2_-T1_)/pnt_bisec),4)

# Print Results==============================================
r = {'Participant': [name_file], 'BP': [pnt_bisec], 'LD': [lim_dif_], 'WR':[WR]}
Results = pd.DataFrame(data=r)
print("\nResultados:\n")
display(Results)

# GRAPH COMMANDs=============================================
print("\nPlot:\n")

# Import seaborn
import seaborn as sns

# Apply the default theme
sns.set_theme()
sns.set_style("ticks")


rng = np.arange(time_stm[0],time_stm[-1],0.1)
pnt_bisec = round(estParms[1],4)

plt.figure(figsize = (4,3),dpi = 200)
plt.plot(rng, yOut, label="Curve-fit [Max Likelihood]", linestyle ='-',lw = 3,color='k')
plt.plot(time_stm, perc_long, color='k', marker='o',linestyle = '',ms = 7,markerfacecolor = 'w')
plt.ylabel('P(Long)',fontsize = 10)
plt.xlabel('Duration (ms)',fontsize = 10)
plt.ylim(-0.1,1.1)
plt.yticks(np.arange(0, 1.1, step=0.25))
plt.grid()

#plt.savefig('BISSEC_{0}'.format(name_file), transparent=True,bbox_inches='tight',format='png')
plt.show()